# รายงานนี้เป็นส่วนหนึ่งของวิชา DSI 206 Multimedia Representation Management

**รายชื่อสมาชิก**

1. นางสาว ญาณิศา จินตนไชยวัฒน์ 6109656287
2. นางสาวณัฐชยา ฉันเฟื่องฟู      6109656493
3. นางสาวอัญวีณ์ พันธ์บูรณานนท์   6109656048

# **1. Data Exploration**

> ขั้นตอนแรก ต้องทำการสำรวจและวิเคราะห์ข้อมูลว่า data ของเรามีอะไรบ้าง สามารถนำข้อมูลส่วนใดไปใช้งานได้ 

> และมีส่วนใดที่ต้องทำการ clean เพื่อทำให้ model ที่ได้นั้นมีประสิทธิภาพมากที่สุด

**- Import Library**

> นำเข้า library ที่ต้องใช้งาน

In [ ]:
# linear algebra
import numpy as np 
# data processing
import pandas as pd 
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt 
import seaborn as sns
# text processing libraries
import string
# regular expression
import re
# natural language processing
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
# scikit-learn
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression

**- Reading Dataset**
> นำเข้าข้อมูลและเรียกดูข้อมูลหัวคอลัมน์ทั้ง Train และ Test data

In [ ]:
# Load data
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sub_sample = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

train.head()

In [ ]:
test.head()

**Columns ทั้งหมดประกอบด้วย 5 columns คือ**
1. **id** [ unique identifier ] เลขลำดับ id ของผู้ใช้
2. **keyword** [ from that tweet ] คีย์เวิร์ดที่อยู่ใน tweet 
3. **location** [ that tweet sent from ] พิกัดของผู้ใช้งาน
4. **text** [ of a tweet ] ข้อความในทวีต
5. **target** - the label we want to predict [ 0 = non-disaster , 1 = disaster ] 

> ซึ่งใน test data จะไม่มี column target เพื่อใช้ทดสอบ model 

**- Check data shape of each data set**

In [ ]:
print('Training Data Shape',train.shape)

Training Data มี **5 columns**** (id , keyword , location , text , target) , และมี **7613 rows**

In [ ]:
print('Testing Data Shape',test.shape)

Testing Data มี **4 columns** (id , keyword , location , text ) , และมี **3263 rows**

In [ ]:
print('Sub Sample Data Shape',sub_sample.shape)
sub_sample.head()

Sub Sample มี **2 columns** (id , target) , และมี **3263 rows**


**- Check Class Distribution of two classes (0 and 1) using Train Data**

* ตรวจสอบจำนวน Target

> 0 เป็นทวีตที่ไม่เกี่ยวข้องกับภัยพิบัติ และ
> 1 เป็นทวีตที่เกี่ยวข้องกับภัยพิบัติ

In [ ]:
train['target'].value_counts()

* สร้างตารางพื่อเปรียบเทียบจำนวนทวีตที่เกี่ยวข้องและไม่เกี่ยวข้องกับภัยพิบัติ

In [ ]:
temp = train.groupby('target').count()['text'].reset_index()
temp['label'] = temp['target'].apply(lambda x : 'Disaster Tweet' if x==1 else 'Non Disaster Tweet')
temp

> จะเห็นว่าทวีตที่เกี่ยวข้องกับภัยพิบัติ (1) มีจำนวนน้อยกว่า ทวีตไม่เกี่ยวข้องกับภัยพิบัติ (0)

* Graph เปรียบเทียบจำนวนข้อความที่เป็น disaster กับ non-disaster

In [ ]:
sns.barplot(train['target'].value_counts().index,train['target'].value_counts()
            ,palette='Spectral')
plt.title('Comparing disaster tweets and non disaster tweets',fontsize=15)

**- Counting Number of Missing Values**

* check แต่ละคอลัมน์ของ Train data ว่ามี missing values จำนวนเท่าใด


In [ ]:
train.isnull().sum()

* check แต่ละคอลัมน์ของ Test data ว่ามี missing values จำนวนเท่าใด


In [ ]:
test.isnull().sum()

**- Target Distribution in ' Keyword '**

* ตรวจสอบว่า Train และ Test มีจำนวน keyword ที่ไม่ซ้ำกันทั้งหมดเท่าใด ใน train data

In [ ]:
train.keyword.nunique(),test.keyword.nunique()

* 10 keyword ที่มีจำนวนมากที่สุดใน Train data



In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(9,6))
# Bar chart showing amount of keywords values
sns.barplot(y=train['keyword'].value_counts()[:10].index,
            x=train['keyword'].value_counts()[:10])
# Add title
plt.title(' Top 10 Keyword ') 
# Add label for x axis
plt.xlabel('COUNT')
# Add label for y axis
plt.ylabel('KEYWORD')
# Rotate the label text for hotizontal axis
plt.xticks(rotation=90) 

* Top 10 Keyword ที่เกี่ยวภัยพิบัติ และที่ไม่เกี่ยวกับภัยพิบัติ

In [ ]:
# create variables a,b (disaster , non-disaster)
a = train[train.target==1].keyword.value_counts().head(10)
b = train[train.target==0].keyword.value_counts().head(10)
# Set the width and height of the figure
plt.figure(figsize=(13,5))
# Bar chart showing amount of disaster keywords values
plt.subplot(121)
sns.barplot(a, a.index, color='orange')
# Add title
plt.title('Top keywords for disaster tweets')
# Bar chart showing amount of non-disaster keywords values
plt.subplot(122)
sns.barplot(b, b.index, color='pink')
# Add title
plt.title('Top keywords for non-disaster tweets')
# display a graph 
plt.show()

**Target Distribution in ' location '**

* ตรวจสอบว่า Train และ Test มีจำนวน keyword ที่ไม่ซ้ำกันทั้งหมดเท่าใด ใน test data

In [ ]:
train.location.nunique(),test.location.nunique()

* 10 location ที่มีจำนวนสูงที่สุดใน Train data

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(9,6))
# Bar chart showing amount of location values and groups the top 10 location
sns.countplot(y=train.location, order = train.location.value_counts().iloc[:10].index)
# Add title
plt.title('Top 10 locations')
# display a graph 
plt.show()

* ทำการแทนที่ชื่อของรัฐต่างๆให้อยู่ในรูปแบบชื่อประเทศ 

> ให้ข้อมูลของ Location แสดงผลเป็นชื่อประเทศ

In [ ]:
# Replacing the ambigious locations name with Standard names
train['location'].replace({'United States':'USA',
                           'New York':'USA',
                            "London":'UK',
                            "Los Angeles, CA":'USA',
                            "Washington, D.C.":'USA',
                            "California":'USA',
                             "Chicago, IL":'USA',
                             "Chicago":'USA',
                            "New York, NY":'USA',
                            "California, USA":'USA',
                            "FLorida":'USA',
                            "Nigeria":'Africa',
                            "Kenya":'Africa',
                            "Everywhere":'Worldwide',
                            "San Francisco":'USA',
                            "Florida":'USA',
                            "United Kingdom":'UK',
                            "Los Angeles":'USA',
                            "Toronto":'Canada',
                            "San Francisco, CA":'USA',
                            "NYC":'USA',
                           "Seattle":'USA',
                            "Earth":'Worldwide',
                            "Ireland":'UK',
                            "London, England":'UK',
                            "New York City":'USA',
                            "Texas":'USA',
                            "London, UK":'UK',
                            "Atlanta, GA":'USA',
                            "Mumbai":"India"},inplace=True)

sns.barplot(y=train['location'].value_counts()[:5].index,x=train['location'].value_counts()[:5])

* เปรียบเทียบจำนวนตัวอักษรทวีต ระหว่างทวีตที่เกี่ยวข้องกับภัยพิบัติและไม่เกี่ยวข้องกับภัยพิบัติ 

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=train[train['target']==1]['text'].str.len()
ax1.hist(tweet_len,color='red')
ax1.set_title('disaster tweets')

tweet_len=train[train['target']==0]['text'].str.len()
ax2.hist(tweet_len,color='blue')
ax2.set_title('Not disaster tweets')
fig.suptitle('Characters in tweets',fontsize=20)

plt.show()

> จากกราฟแสดงให้เห็นว่า จำนวนทวีตที่เกี่ยวข้องกับภัยพิบัติมีจำนวนตัวอักษรในทวีตอยู่ในช่วง 120-140 
เช่นเดียวกันกับทวีตที่ไม่เกี่ยวข้องกับภัยพิบัติ

* เปรียบเทียบจำนวนคำในทวีต ระหว่างทวีตที่เกี่ยวข้องกับภัยพิบัติและไม่เกี่ยวข้องกับภัยพิบัติ 

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=train[train['target']==1]['text'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='red')
ax1.set_title('disaster tweets')
tweet_len=train[train['target']==0]['text'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='blue')
ax2.set_title('Not disaster tweets')
fig.suptitle('Words in a tweets',fontsize=20)
plt.show()

> จากกราฟแสดงให้เห็นว่า ทวีตที่เกี่ยวข้องกับภัยพิบัติมีจำนวนคำอยู่ในช่วง 17-18 คำ มากที่สุด และทวีตที่ไม่เกี่ยวข้องกับภัยพิบัติมีจำนวนคำอยู่ในช่วง 18-19 คำ มากที่สุด

# 2. Data Cleaning

> จากการที่ได้สำรวจข้อมูลแล้ว ขั้นตอนต่อไป คือ การทำความสะอาดข้อมูล 

> โดยการลบ data ส่วนที่ไม่เกี่ยวข้องหรือส่วนที่ไม่จำเป็นออกไป ให้ข้อมูลมีคุณภาพมากที่สุด เพื่อเตรียมพร้อมในการทำ model 

* แทนที่ช่องว่าง null ด้วยคำว่า 

> no_location ในคอลัมน์ location

> no_keyword ในคอลัมน์ Keyword

In [ ]:
for col in ['keyword', 'location']:
    train[col] = train[col].fillna(f'no_{col}')
for col in ['keyword', 'location']:
    test[col] = test[col].fillna(f'no_{col}')

In [ ]:
#train.head()
test.head()

* ทำการ Clean data โดยใช้ Regular Expression 

> ทำตัวอักษรให้เป็นตัวพิมพ์เล็กทั้งหมด

> ลบเครื่องหมายต่างๆที่ไม่ต้องการ เช่น !€@%#*&~ รวมไปถึง URL , HTML , ขึ้นบรรทัดใหม่ , เครื่องหมายวรรคตอน , คำที่มีตัวเลขคั่น 

In [ ]:
# Applying a first round of text cleaning techniques

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'<.*?>',' ' ,text)
    
    return text

# Applying the cleaning function to both test and training datasets
train['text'] = train['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

# Let's take a look at the updated text
train['text'].head()

* ทำการแบ่งคำ โดยใช้ RegexpTokenizer 

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

# Tokenizing the training and the test set
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x))
test['text'] = test['text'].apply(lambda x: tokenizer.tokenize(x))
train['text'].head()


* ทำการลบ stop words ในภาษาอังกฤษ 

> เช่น the, a, at, for, above, on, is, all เป็นต้น โดยใช้ nltk library
เนื่องจากไม่มีความหมายในการใช้วิเคราะห์ 

In [ ]:
def remove_stopwords(text):
    """
    Removing stopwords belonging to english language
    
    """
    words = [w for w in text if w not in stopwords.words('english')]
    return words


    train['text'] = train['text'].apply(lambda x : remove_stopwords(x))
    test['text'] = test['text'].apply(lambda x : remove_stopwords(x))

* ทำการ remove emoji 

> โดยการแทน emoji เป็นรหัสในทุกๆหมวดหมู่ และลบโดยใช้ library re

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

* เปลี่ยนคำในทวีตให้อยู่ในรูปเดียวกันโดยการทำ Stemmimg

> คือการตัดคำลงท้ายออก (เช่น s, es, ed, ing) 

> stem สามารถเขียนได้หลายรูป stemmimg, stemmed,stems เราจะทำการแปลงให้มันอยู่ในรูปเดียวคือ stem

In [ ]:
def stemming(words):
     ps=PorterStemmer()
     return [ps.stem(word) for word in words]
train['text']=train['text'].apply(lambda x: stemming(x))
test['text']=test['text'].apply(lambda x: stemming(x))

* Lemmatizing คือการทำให้คำที่อยู่ในรูปแบบต่างๆ แปลงกลับมาอยู่ในรูปปกติ root word

> เช่น Feet เป็น Foot //
wolves เป็น wolf //
is,am,are เป็น be

In [ ]:
def lemmatizing(words):
            lemmatizer =WordNetLemmatizer()
            return [lemmatizer.lemmatize(word) for word in words]
train['text']=train['text'].apply(lambda x: lemmatizing(x))
test['text']=test['text'].apply(lambda x: lemmatizing(x))

* เอาคำในประโยคมาเรียงต่อกัน ด้วยการ join word 

In [ ]:
def final_text(words):
     return ' '.join(words)
train['text']=train['text'].apply(lambda x:final_text(x))
test['text']=test['text'].apply(lambda x:final_text(x))
train.head(10)

In [ ]:
test.head(10)

# 3. Modeling


# *> Bag-of-word*

In [ ]:
count_vectorizer = CountVectorizer()
train_vectors = count_vectorizer.fit_transform(train['text'])
test_vectors = count_vectorizer.transform(test["text"])


* ทำ bag-of-word โดยการนับว่ามีคำๆนั้นปรากฎทั้งหมดกี่ครั้งในประโยค แล้วนำคำศัพท์ไปใส่ด้วย function CountVectorizer ที่จะแปลงข้อความเป็น vector เพื่อลดมิติของข้อมูล และนำไปคำนวนได้ 
โดยการนำกลุ่มของ token มีสร้างเป็น matrix โดยใช้กลุ่มของคำที่มีเป็นตัวอ้างอิง คำที่มีในประโยคจะถูกตั้งค่าเป็น 1 คำที่ไม่มีจะเป็น 0 

> เช่น มีกลุ่มของคำ [“This”, “is”, “am”, “are”, “a”, “be”, “test”, “word”, “sentence”] 

> ประโยค “This is a test sentence” จะแปลงเป็น matrix ได้ดังนี้ [1, 1, 0, 0, 1, 0, 1, 0 ,1]

* **ปัญหาที่พบ** คือ ทวีตที่ยาวจะมีน้ำหนักของคำเยอะกว่าทวีตที่สั้นกว่า ทำให้ผลลัพธ์ที่ได้ไม่แม่นยำ จึงเลือกใช้เป็น TF-IDF แทน เพื่อแก้ไขปัญหานี้

# *> TF-IDF*

In [ ]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_vectors = tfidf.fit_transform(train['text'])
test_vectors = tfidf.transform(test["text"])

* ทำ bag-of-word โดยการใช้ TfidfVectorizer ที่เป็น function ใช้แปลงข้อความเป็น vector เช่นเดียวกันกับ CountVectorizer

> เพื่อเพิ่มความถูกต้องในการนับ โดยการใช้ Term Frequency ของจำนวนครั้งที่คำนั้นปรากฏในทวีต คำนวณโดยใช้สูตร TF = จำนวนครั้งคำนั้นๆ/จำนวนคำทั้งหมดในแต่ละทวีต

> แล้วจึงนำมาคูณกับ Inverse Document Frequency แล้วทำการ take log 
> วิธีนี้จึงเป็นวิธีการหาความสำคัญของคำ ที่ใช้แนวคิดว่ายิ่งคำนั้นปรากฏน้อยจะยิ่งมีความสำคัญมาก โดยนำค่า TF กับ IDF มาคูณกัน

# *> ทำ Text Classification โดยการใช้ Logistic Regression และ Cross Validation*

**Cross Validation**
* แบ่งข้อมูลเรียนรู้ออกเป็น k ชุดเท่า ๆ กัน และทำการคำนวน error จำนวน k รอบ
* ใช้ข้อมูลส่วนที่เหลือ (k-1 ชุด) เพื่อทำการสร้าง model
* เก็บข้อมูลที่แบ่งไว้ 1 ชุด เพื่อทำการ evaluate
* วนทำซ้ำจนข้อมูลทุกส่วนถูกนำมาทดสอบ

เราได้เลือกใช้ Logistic Regression ในการ predict probability โดยกำหนดค่า parameter C เท่ากับ 0.9 และเก็บในตัวแปร clf จากนั้นจึงคำนวนหาค่า error ของแต่ละรอบการคำนวน

In [ ]:
clf = LogisticRegression(C=0.9,max_iter=1000,penalty='l2')
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=7, scoring="f1")
scores

นำผลที่ได้มา fit เข้ากับ train_vector และ column target ของ train

In [ ]:
clf.fit(train_vectors, train["target"])

นำ model ที่ได้มา predict กับ test_vectors และนำมา check กับเฉลย คือ target ในไฟล์ sample_submission จากนั้นจึงสร้างไฟล์ csv เพื่อทำการ submit ผลที่ได้ใน kaggle 

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = clf.predict(test_vectors)
sample_submission.to_csv("submission.csv", index=False)

# References



> sahib singh ,"NLP Starter for Beginners", https://www.kaggle.com/sahib12/nlp-starter-for-beginners

> Bavalpreet ,"NLP with Disaster Tweets" , https://www.kaggle.com/bavalpreet26/nlp-with-disaster-tweets